In [1]:
import os 
from dotenv import find_dotenv, load_dotenv # type: ignore
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.environ['GROQ_API_KEY']

In [2]:
from langchain_community.document_loaders import TextLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore
from langchain_groq import ChatGroq # type: ignore

Loading the data

In [4]:
loader = TextLoader('D:/ragspecific/evaluation/mentalhealth.txt')
document = loader.load()

Chunking the data

In [5]:
text_spilitter  = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)

In [6]:
chunks = text_spilitter.split_documents(document)

In [7]:
chunks

[Document(metadata={'source': 'D:/ragspecific/evaluation/mentalhealth.txt'}, page_content='Personality Scores:\n1. Agreeableness'),
 Document(metadata={'source': 'D:/ragspecific/evaluation/mentalhealth.txt'}, page_content="1. Agreeableness\nLow Agreeableness: A low score in agreeableness, such as 0.05, suggests that an individual may exhibit skepticism towards others' intentions, potentially leading to a more competitive or critical demeanor rather than cooperative. Such individuals may prioritize their own interests over others and might struggle with showing empathy. They can be perceived as tough, critical, and uncompromising, often engaging in more confrontational or competitive interactions."),
 Document(metadata={'source': 'D:/ragspecific/evaluation/mentalhealth.txt'}, page_content='High Agreeableness: Conversely, a high score indicates a person who is generally compassionate, cooperative, and eager to help others, often even at a personal cost. High agreeableness is characterize

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8428\801722267.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\ProgramData\anaconda3\envs\prac\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain_community.vectorstores import FAISS # type: ignore
vectordb = FAISS.from_documents(chunks, embeddings_model)

In [11]:
retriever = vectordb.as_retriever()

Define llm

In [12]:
llm = ChatGroq(model = "mixtral-8x7b-32768")

define prompt

In [13]:
template = """  
You are a helpful mentalhealth assitant,
Question : {question},
context : {context},
Answer:

"""

In [14]:
from langchain.prompts import ChatPromptTemplate # type: ignore
from langchain.schema.runnable import RunnablePassthrough # type: ignore
from langchain.schema.output_parser import StrOutputParser # type: ignore

In [15]:
prompt = ChatPromptTemplate.from_template(template)

setting up rag pipeline

In [16]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

In [17]:
chain.invoke('what is low agreeableness')

"Low agreeableness refers to a personality trait where an individual exhibits skepticism towards others' intentions, potentially leading to a more competitive or critical demeanor. Individuals with low agreeableness tend to prioritize their own interests over others and might struggle with showing empathy. They can be perceived as tough, critical, and uncompromising, often engaging in more confrontational or competitive interactions. In contrast, a high score in agreeableness indicates a person who is generally compassionate, cooperative, and eager to help others, often even at a personal cost."

In [18]:
questions = [
    "I don't like people who are emotional.",
    "I find it hard to open up to others.",
    "I always need to be doing something productive.",
    "I prefer sticking to my routines than trying out new things.",
    "I often feel misunderstood by others.",
    "I'm always the life of the party, but sometimes it's exhausting.",
    "I get anxious when plans change last minute.",
    "I love learning new things, but I rarely finish what I start.",
    "I don't enjoy socializing much; I prefer my own company.",
    "I often feel overwhelmed by my emotions."
]

ground_truths = [
    "It's okay to feel that way. We all have different comfort levels with emotions. Would you like to explore ways to understand or cope with this feeling?",
    "It's completely understandable to find it challenging to open up, especially when you're naturally empathetic yet introspective. Your kindness and thoughtfulness are clear strengths. If you're comfortable, we can explore small, safe steps to gradually share your feelings, ensuring you feel supported and secure throughout.",
    "Your drive for productivity is admirable, and it's great that you're open to new experiences. Balancing activity with moments of rest can enhance your well-being and creativity. Would you be interested in exploring strategies to integrate meaningful pauses into your routine, ensuring you're recharged and ready for your next adventure?",
    "Sticking to routines can provide a comforting sense of stability and control. It's okay to prefer the familiar; it's about finding a balance that makes you feel secure and fulfilled. If you're ever curious, we could gently explore small changes within your comfort zone to add a little variety, without overwhelming your sense of order. How does that feel?",
    "Feeling misunderstood can be really tough, especially when you're open to sharing your thoughts and connecting with others. It's important to recognize your feelings are valid. Together, we can explore ways to express your ideas and emotions in a way that feels authentic to you, while also considering strategies to foster understanding and connection. What are your thoughts on this approach?",
    "It's wonderful that you bring so much energy and joy to social gatherings. It's also important to listen to your own needs and allow yourself time to recharge. Finding a balance between socializing and personal time can keep your spirits high without feeling overextended. Would you like to explore ways to ensure you're taking care of yourself while still enjoying the social scene?",
    "It's natural to feel unsettled by sudden changes, especially when you value order and reliability. Let's explore strategies that can help you feel more at ease during unexpected situations, ensuring you have tools to navigate these moments with confidence. How does that sound?",
    "Your passion for exploring new ideas is wonderful! It's common to find it challenging to stick with one thing when there's so much to discover. Would you be interested in discussing ways to balance your love for learning with techniques to help you complete what you start?",
    "Preferring your own company is completely okay, and it's important to honor what feels right for you. If you're ever looking for ways to gently expand your comfort zone, without compromising your need for solitude, I'm here to support you. Shall we explore this together?",
    "Feeling overwhelmed by emotions can be intense. It's courageous to acknowledge these feelings. Together, we can explore grounding techniques and coping strategies that can help you navigate these moments with greater ease. Would you like to start this journey?"
]
answer = []
content = []

inference

In [19]:
for query in questions:
    answer.append(chain.invoke(query))
    content.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8428\1021677130.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  content.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


In [20]:
answer

["It sounds like you have a high level of neuroticism, which can make you more sensitive to emotional expressions and potentially overwhelmed by them. Additionally, you mentioned that you struggle to express your emotions openly and often feel disconnected from your emotions. These experiences could be related to a fear of vulnerability or rejection. It's important to remember that emotions are a natural and important part of the human experience. If you're finding it difficult to manage your emotions or are experiencing distress because of them, it might be helpful to speak with a mental health professional. They can provide guidance and support to help you better understand and navigate your emotional experiences.",
 "It's understandable that you find it hard to open up to others, given your high agreeableness, low extraversion, and moderate neuroticism. Your kind and empathetic nature, along with your emotional sensitivity and introverted tendencies, can make it challenging to expre

In [21]:
content

[['User:  "I don\'t like people who are emotional."\nPersonality scores:\nAgreeableness:  0.05\nExtraversion:  0.02\nOpenness:  0.02\nConscientiousness:  0.02\nNeuroticism:  0.89\nContext: Given the personality traits, the user may express disdain for emotional people due to their own discomfort with emotions and a preference for more logical or detached interactions. Their high neuroticism could make them more sensitive to emotional expressions, which they might find overwhelming or unnecessary.',
  'User: "I don\'t like people who are overly emotional."\nPersonality Scores:\nAgreeableness: 0.05\nExtraversion: 0.02\nOpenness: 0.02\nConscientiousness: 0.02\nNeuroticism: 0.89\nContext: Given the personality traits, the user may express disdain for overly emotional people due to their own discomfort with emotions and a preference for more logical or detached interactions. Their high neuroticism could make them more sensitive to emotional expressions, which they might find overwhelming or

### Please Go Ragas Documentation for more information

Metrics : https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/?h=metrics


For example context precision we required input (query),response and retrieved_contexts 

check more detail here

[Context_precision](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/context_precision/)


same apply to other metrics , check what is needed to calculate perticular metrics and how it calculated

In [22]:
data = {
    "question": questions,
    #"ground_truths": ground_truths, 
    "answer": answer,
    #"context": content,
    "retrieved_contexts" : content, # retrive context that is content is stored as retrieved_contexts to calulate matrics
    "reference": ground_truths # ground truth is stored as referene as ragas use this key to calculate metrics 
}

In [23]:
from datasets import Dataset # type: ignore
dataset = Dataset.from_dict(data)

In [24]:
dataset

Dataset({
    features: ['question', 'answer', 'retrieved_contexts', 'reference'],
    num_rows: 10
})

In [25]:
from ragas import evaluate # type: ignore

from ragas.metrics import ( # type: ignore
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness
)


In [26]:
results = evaluate(dataset=dataset,
                   metrics=[faithfulness,answer_relevancy,context_recall,context_precision],
                   embeddings=embeddings_model, # type: ignore
                   llm = llm,
                   batch_size = 2)


Evaluating: 100%|██████████| 40/40 [21:44<00:00, 32.62s/it]


In [27]:
results

{'faithfulness': 0.7819, 'answer_relevancy': 0.4626, 'context_recall': 0.7314, 'context_precision': 0.9806}